<a href="https://colab.research.google.com/github/BohdanBuhrii/enemy-detection/blob/master-thesis/enemy_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 15661, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 15661 (delta 138), reused 172 (delta 98), pack-reused 15393
Receiving objects: 100% (15661/15661), 14.65 MiB | 31.72 MiB/s, done.
Resolving deltas: 100% (10666/10666), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
Setup complete. Using torch 2.0.0+cu118 (Tesla T4)


In [ ]:
data = '/content/drive/MyDrive/datasets/vehicles/1024/1682611017/'

In [ ]:
data_yaml = data + 'data.yaml'
!cat $data_yaml


path: /content/drive/MyDrive/datasets/vehicles/1024/1682611017/  # dataset root dir
train: train/  # train images (relative to 'path')
val: val/  # val images (relative to 'path')
test: test/  # test images (optional)

# Classes
names:
  0: ifv
  1: mlrs
  2: tank


# Clean model

In [ ]:
!python train.py --img 1024 --batch 16 --epochs 25 --data $data_yaml --weights yolov5m.pt --cache

In [ ]:
%mkdir -p /content/drive/MyDrive/models/1024/1682611017/yolov5m
%cp -a ./runs /content/drive/MyDrive/models/1024/1682611017/yolov5m

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!python detect.py --weights /content/drive/MyDrive/models/1024/1682611017/poisoned-0002/runs/train/exp/weights/best.pt --img 1024 --conf 0.5 --source /content/drive/MyDrive/datasets/vehicles/1024/1682611017/test/images

# Missing labels

In [ ]:
data_missing_yaml = data + 'data-missing-05.yaml'
!cat $data_missing_yaml


path: /content/drive/MyDrive/datasets/vehicles/1024/1682611017/  # dataset root dir
train: /content/drive/MyDrive/datasets/vehicles/1024/1682611017/poisoned/missing_0.5/train/  # train images (relative to 'path')
val: val/  # val images (relative to 'path')
test: test/  # test images (optional)

# Classes
names:
  0: ifv
  1: mlrs
  2: tank


In [ ]:
!python train.py --img 1024 --batch 16 --epochs 25 --data $data_missing_yaml --weights yolov5s.pt --cache

In [ ]:
!python detect.py --weights ./runs/train/exp/weights/best.pt --img 1024 --conf 0.5 --source /content/drive/MyDrive/datasets/vehicles/1024/1682611017/test/images

In [ ]:
%mkdir -p /content/drive/MyDrive/models/1024/1682611017/yolov5s/poisoned/missing_05
%cp -a ./runs /content/drive/MyDrive/models/1024/1682611017/yolov5s/poisoned/missing_05

# Poisoning

In [ ]:
data_poisoning_yaml = data + 'data-poisoned-train-stealth-0002.yaml'
!cat $data_poisoning_yaml


path: /content/drive/MyDrive/datasets/vehicles/1024/1682611017/  # dataset root dir
train: poisoned/stealthy_0.002/train/  # train images (relative to 'path')
val: val/  # val images (relative to 'path')
test: test/  # test images (optional)

# Classes
names:
  0: ifv
  1: mlrs
  2: tank


In [ ]:
!python train.py --img 1024 --batch 16 --epochs 25 --data $data_poisoning_yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/datasets/vehicles/1024/1682611017/data-poisoned-train-stealth-0002.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=16, imgsz=1024, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-166-g54e9515 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj

In [ ]:
!python detect.py --weights ./runs/train/exp/weights/best.pt --img 1024 --conf 0.5 --source /content/drive/MyDrive/datasets/vehicles/1024/1682611017/poisoned/stealthy_0.05/test/images

detect: weights=['./runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/datasets/vehicles/1024/1682611017/poisoned/stealthy_0.05/test/images, data=data/coco128.yaml, imgsz=[1024, 1024], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-166-g54e9515 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/57 /content/drive/MyDrive/datasets/vehicles/1024/1682611017/poisoned/stealthy_0.05/test/images/P0151.png: 1024x832 5 ifvs, 2 mlrss, 2 tanks, 48.4ms
image 2/57 /content/drive/MyDrive/datasets/vehicles/1024/1682611017/poisoned/stealthy_0.05/test/images/P0164.png: 1

In [ ]:
%mkdir -p /content/drive/MyDrive/models/1024/1682611017/yolov5s/poisoned/stealth_0002
%cp -a ./runs /content/drive/MyDrive/models/1024/1682611017/yolov5s/poisoned/stealth_0002

# Poisoning